Importing all the libraries to be used in code

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

Reading the Text file on which we will train our RNN model


In [13]:
text=open("/content/sonnets.txt").read()
text=text.lower()

Creating a dictionary of characters which are present in the text file

In [14]:
chars=sorted(list(set(text)))

Making two lists:
1> from characters to index
2> from index to characters

In [15]:
chars_to_ix={ch:i for i,ch in enumerate(chars)}
ix_to_chars={i:ch for i,ch in enumerate(chars)}


**Generating Two arrays of type text, one for input(sequences) and one for output(target)**

Seq_length determine the time steps which we use in RNN LSTM block

We use a many to one prediction model:
for example text is "I am awesome" and seq_length is 4

"i am" produces " "
" am " produces "a"
"am a" produces "w"
"m aw" produces "e"
... so on






In [20]:
seq_length=100
step=1

sequences=[]
targets=[]
for i in range(0,len(text)-seq_length,step):
  sequences.append(text[i:i+seq_length])
  targets.append(text[i+seq_length])
print(len(sequences))

95560


Vectorizing the data:
creating input matrix X and output matrix Y from sequences and target respectively

In [21]:
X=np.zeros((len(sequences),seq_length,len(chars)))
Y=np.zeros((len(sequences),len(chars)))

for i in range(len(sequences)):
  sentence=sequences[i]
  for j,ch in enumerate(sentence):
    X[i,j,chars_to_ix[ch]]=1
  Y[i,chars_to_ix[targets[i]]]=1

Creating the a simple model:Adding LSTM layer and a Dense layer

In [23]:
model=Sequential()
model.add(LSTM(128,input_shape=(seq_length,len(chars))))
model.add(Dense(len(chars),activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               86016     
_________________________________________________________________
dense (Dense)                (None, 39)                5031      
Total params: 91,047
Trainable params: 91,047
Non-trainable params: 0
_________________________________________________________________


Compiling the model

In [25]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

Sample function defined for sampling the prediction

In [37]:
def sample(preds,temperature=1.0):
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds)/temperature
  exp_preds=np.exp(preds)
  preds=exp_preds/np.sum(exp_preds)
  probas=np.random.multinomial(1,preds,1)
  return np.argmax(probas)

Finally text will be generated by calling the generate_output function, with total characters: seq_length+600

In [38]:
import sys
import os

def generate_output(temp):
  start_idx=np.random.randint(0,len(text)-seq_length-1)
  output_text=text[start_idx:start_idx+seq_length]
  sys.stdout.write(output_text)

  for i in range(600):
    sampled=np.zeros((1,seq_length,len(chars)))
    for i,ch in enumerate(output_text):
      sampled[0,i,chars_to_ix[ch]]=1
    probs=model.predict(sampled,verbose=0)[0]
    pred_idx=sample(probs,temperature=temp)
    next_char=ix_to_chars[pred_idx]

    output_text+=next_char
    output_text=output_text[1:]

    sys.stdout.write(next_char)
    sys.stdout.flush()


Training the model

In [35]:
model.fit(X,Y,batch_size=128,epochs=20)

Epoch 1/20
747/747 [==============================] - 9s 11ms/step - loss: 1.4512
Epoch 2/20
747/747 [==============================] - 9s 11ms/step - loss: 1.4357
Epoch 3/20
747/747 [==============================] - 9s 11ms/step - loss: 1.4218
Epoch 4/20
747/747 [==============================] - 9s 11ms/step - loss: 1.4081
Epoch 5/20
747/747 [==============================] - 9s 11ms/step - loss: 1.3943
Epoch 6/20
747/747 [==============================] - 8s 11ms/step - loss: 1.3811
Epoch 7/20
747/747 [==============================] - 9s 11ms/step - loss: 1.3693
Epoch 8/20
747/747 [==============================] - 9s 11ms/step - loss: 1.3561
Epoch 9/20
747/747 [==============================] - 9s 11ms/step - loss: 1.3439
Epoch 10/20
747/747 [==============================] - 9s 11ms/step - loss: 1.3320
Epoch 11/20
747/747 [==============================] - 9s 11ms/step - loss: 1.3192
Epoch 12/20
747/747 [==============================] - 9s 11ms/step - loss: 1.3075
Epoch 13/20
7

Finally the output

In [39]:
generate_output(0.77)

 of worth, what worth in you doth grow.
this silence for my sin you did impute,
which shall be most and winteing her graces:
the ounding sweet invention stranged dears.

lxxxii.

o, me, the congol, that gunds and how can pine
which, i seem but in the worse, which you were,
hath leath, your come than hath me rememmer'd
age streast,, when thou says are now devire?
and me, yourself in days of your thee reads,
if such since my still shall me a true,
whilst i as youth, when i time which new;
but those bained that when to straight,
  age in this pity of thy sun mast from thee,
and youtherome that which not figst mine eyes,
though in thus do i as is sovel;
but dear thou, full negear still reseet sa